# Predição do placar de partidas do futebol

Este notebook tem como objetivo desenvolver e avaliar modelos de machine learning para prever o placar de partidas de futebol, utilizando como base dados históricos de partidas e estatísticas dos times. O foco está na criação de um modelo que preveja quantos gols cada time (da casa e visitante) marcará em uma partida.

## Objetivo

O objetivo deste notebook é estruturar a hipótese de maneira clara e realizar análises que possam confirmá-la ou refutá-la, utilizando métodos estatísticos e exploratórios, culminando na predição do placar das partidas de futebol.


## Como Usar Este Notebook

1. **Configuração do Ambiente**:
   - **Google Colab**: No Google Colab, faça o upload das tabelas para o Google Drive e monte o drive no notebook.
   - **Localmente**: Se for rodar localmente, ajuste os caminhos dos arquivos ou coloque-os no mesmo diretório do notebook.

2. **Instalação de Dependências**:
   - Instale todas as bibliotecas necessárias com o comando:
 	```python
 	!pip install -r requirements.txt
 	```

3. **Execução do Notebook**:
   - Execute os notebooks relacionados à exploração e ao pré-processamento das tabela `Matches`:
  	- [Notebook - Exploração - Matches](https://github.com/Inteli-College/2024-2A-T14-IN03-G05/blob/3998e6d5984b905ac4588992ce6c31563b3edc0c/notebooks/exploracao/matches_explorado.ipynb)
  	- [Notebook - Pre Processamento - Matches](https://github.com/Inteli-College/2024-2A-T14-IN03-G05/blob/c58b4300d4dd212937e7e43199f2be18878b4b32/notebooks/pre_processamento/matches_processado.ipynb)


## Nesse Notebook Será Abordado

1. **Modelagem para o problema**:
   - Apresentar o modelo e para que estamos utilizando;
   - Métricas relacionadas ao modelo - avaliação de desempenho;
   - Hiperparâmetros - aplicação de GridSearch.

## Importando bibliotecas

Aqui é importado as dependências necessárias para a execução do projeto.

In [36]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE

## Carregando o arquivo CSV
O arquivo matches_tratado.csv é carregado e armazenado na variável df_matches. Este dataset contém as informações das partidas e estatísticas relevantes para a modelagem preditiva, como número de gols, cartões, posse de bola, escanteios, entre outros.

In [37]:
df_matches = pd.read_csv('../data/tratado/matches_tratado.csv');

## 1.1 Regressão Logística

### 1.2 Hipótese do Modelo: Times que jogam em casa possuem uma tendência maior de ganharem.

##### Premissa Principal:
O objetivo de previsão desse modelo é que o desempenho passado de um time, medido por diversas métricas estatísticas (como posse de bola, número de chutes a gol, cartões, etc.), ainda assim tem como tendência o time que está jogando em casa ganhar. O modelo utiliza uma combinação de variáveis pré-jogo e dados históricos dos times para calcular essas probabilidades.

##### Justificativa da Premissa:
1. **Apoio da torcida, familiaridade com o campo e ausência de desgastes físicos e mentais devido a viagens** são fatores podem permitir que o time da casa jogue com mais confiança, resultando em vitórias.
2. **Pressão do ambiente adverso e a adaptação ao campo** são fatores que podem prejudicar a performance das equipes, impactando na capacidade de pontuar.

### ***Conclusões:***

O gráfico confirma a hipótese de que os times têm uma média de pontos maior quando jogam em casa, reforçando a ideia de que o fator casa oferece vantagens significativas em termos de desempenho. A familiaridade com o ambiente, parece desempenhar um papel importante na conquista de pontos pelos times mandantes. Já os times visitantes enfrentam maiores dificuldades, o que justifica a menor média de pontos fora de casa.

#### Abordagem Técnica

Nesta parte foi utilizado o modelo de **Regressão Logística** e Random Forest, sendo aplicada para realizar as previsões de futebol. Alguns passos foram: A imputação de valores faltantes, padronização dos dados, divisão dos dados e treinamento do modelo.

- A **Regressão Logística** é um modelo estatístico usado para prever a probabilidade de um evento binário (com dois resultados possíveis) ou multiclasse.
- O **Random Forest** foi utilizado para prever as quantidades de gols dos times em uma partida, com o objetivo de gerar o placar exato e os placares mais prováveis.
- O modelo foi treinado com um conjunto de dados que contém estatísticas de partidas anteriores e diversas métricas de desempenho dos times.

#### Estrutura do Modelo

- **Variáveis Preditoras**: 
  - Estatísticas da equipe da casa e da equipe visitante, incluindo posse de bola, número de chutes a gol, cartões amarelos e vermelhos, escanteios, xG (gols esperados) e faltas cometidas.
  - Dados pré-jogo como Pre-Match PPG (pontos por jogo) e Pre-Match xG (gols esperados).
  
- **Variável Alvo**: Quantidade de gols do time da casa e do time visitante.

#### Previsão Ajustada

O modelo utiliza duas abordagens para prever os resultados das partidas de futebol:

1. **Previsão do Placar Certeiro**:
   - Este modelo é treinado para prever a quantidade exata de gols que o time da casa e o time visitante irão marcar. Utilizando variáveis preditoras, como posse de bola, escanteios, cartões, xG, entre outras, o modelo gera o placar exato esperado para a partida.

2. **Previsão dos Top 3 Placares Prováveis**:
   - O segundo modelo prevê os três resultados mais prováveis para a partida. Baseado nas estatísticas das equipes e em um sistema de variação nos gols preditos, o modelo gera os três placares mais plausíveis que podem ocorrer durante o jogo, oferecendo uma visão mais ampla das possíveis variações.

Essa estrutura permite que  tenha tanto uma previsão certeira quanto uma probabilidade dos cenários mais possíveis para cada partida.

#### Exemplo de Funcionamento

O modelo é capaz de prever resultados de partidas futuras, mesmo que os dados dessas partidas não estejam no conjunto de dados de treino. Nesse caso, ele utiliza **médias históricas** dos times para fazer previsões, garantindo que ainda possa fornecer um placar estimado, com base no desempenho geral dos times, tanto em termos de gols exatos quanto dos três placares mais prováveis.


### Funções para Extração de Features

Duas funções principais são usadas para extrair características essenciais para o modelo:

- **`calcular_gols_recentes(time_encoded, n=5)`**: Calcula a média de gols marcados por um time nas últimas 5 partidas.
- **`calcular_confronto_direto(home_team_encoded, away_team_encoded)`**: Calcula a média de gols nos confrontos diretos entre dois times específicos.

Essas informações são adicionadas ao dataset como novas features: **home_gols_recentes**, **away_gols_recentes** e **historico_confronto_direto**.

### Seleção de Features

Além das features tradicionais, como posse de bola, escanteios e cartões, foram criadas novas variáveis que capturam interações entre os dados e relações mais complexas, ampliando o potencial preditivo do modelo.

As principais features utilizadas no treinamento do modelo são:

1. **Estatísticas pré-jogo (PPG - Pontos por Jogo)**:
   - *Pre-Match PPG (Home)*: Pontos por jogo do time da casa.
   - *Pre-Match PPG (Away)*: Pontos por jogo do time visitante.

2. **Posse de Bola**:
   - *home_team_possession*: Porcentagem de posse de bola do time da casa.
   - *away_team_possession*: Porcentagem de posse de bola do time visitante.
   - *Nova Feature*: *possession_diff* (diferença de posse de bola entre o time da casa e o visitante).

3. **Escanteios**:
   - *home_team_corner_count*: Escanteios do time da casa.
   - *away_team_corner_count*: Escanteios do time visitante.
   - *Nova Feature*: *home_corners_per_possession* e *away_corners_per_possession* (proporção de escanteios por posse de bola).

4. **Cartões Amarelos e Vermelhos**:
   - *home_team_yellow_cards*: Cartões amarelos do time da casa.
   - *away_team_yellow_cards*: Cartões amarelos do time visitante.
   - *home_team_red_cards*: Cartões vermelhos do time da casa.
   - *away_team_red_cards*: Cartões vermelhos do time visitante.
   - *Nova Feature*: *cards_diff* (diferença de cartões entre os times).

5. **Eficiência Ofensiva**:
   - *Nova Feature*: *home_efficiency_offense* e *away_efficiency_offense* (gols por posse de bola para o time da casa e visitante).

6. **Outras Interações Importantes**:
   - *xg_diff*: Diferença entre os gols esperados (xG) dos times.
   - *ppg_x_xg*: Produto da diferença de PPG e xG do time visitante.
   - *xg_x_corner*: Produto da diferença de xG e escanteios.
   - *xg_x_card*: Produto de xG do time visitante e a diferença de cartões.
   - *corner_x_card*: Produto da diferença de escanteios e cartões.
   - *ppg_x_xg_diff*: Produto da diferença de PPG e de xG.
   - *ppg_x_corner*: Produto da diferença de PPG e escanteios.
   - *xg_x_ppg_away*: Produto da diferença de xG e PPG do time visitante.
   - *team_b_xg_x_corner*: Produto do xG do time visitante e a diferença de escanteios.
   - *ppg_x_card*: Produto da diferença de PPG e a diferença de cartões.

Essas features, incluindo interações e variações calculadas entre as estatísticas, foram cuidadosamente escolhidas para melhorar a performance do modelo, capturando as complexas relações entre os dados de desempenho das equipes.

### Divisão de Dados

Os dados foram divididos em:

- *Conjunto de Treinamento*: 80% dos dados históricos, usados para ajustar o modelo.
- *Conjunto de Teste*: 20% dos dados, usados para avaliar o desempenho do modelo.

Essa divisão foi aplicada separadamente para as variáveis dependentes (gols do time da casa e gols do time visitante).

### Treinamento do Modelo Random Forest

Dois modelos de **Random Forest Regressor** foram treinados para prever os gols do time da casa e do time visitante:

- *Modelo para o Time da Casa (`best_rf_home`)*: Responsável por prever a quantidade de gols que o time da casa marcará.
- *Modelo para o Time Visitante (`best_rf_away`)*: Responsável por prever a quantidade de gols que o time visitante marcará.

**Hiperparâmetros Otimizados**: Utilizando técnicas como GridSearchCV e RandomSearch, foram encontrados os melhores hiperparâmetros para cada modelo. Alguns deles incluem:
- *n_estimators*: Número de árvores no modelo.
- *max_depth*: Profundidade máxima permitida para as árvores.
- *min_samples_split*: Número mínimo de amostras necessárias para dividir um nó.
- *min_samples_leaf*: Número mínimo de amostras necessárias em um nó folha.

Esses hiperparâmetros permitiram melhorar o desempenho dos modelos, equilibrando a capacidade de previsão e a prevenção de overfitting.

### Seleção de features

Essas features foram criadas para melhorar a capacidade preditiva do modelo de machine learning, capturando interações importantes entre as estatísticas dos times em uma partida de futebol. Aqui estão breves descrições de cada uma:

1. **Diferença de posse de bola**: Mede a diferença entre a porcentagem de posse de bola do time da casa e do time visitante, indicando o controle de jogo de cada equipe.
   
2. **Proporção de escanteios por posse de bola**: Calcula a eficiência do time ao gerar escanteios em relação ao tempo que manteve a posse de bola, para ambos os times, casa e visitante.
   
3. **Diferença de cartões**: Representa a diferença entre o número de cartões amarelos e vermelhos recebidos pelos times da casa e visitante, refletindo o nível de disciplina ou agressividade em campo.

4. **Eficiência ofensiva (gols por posse de bola)**: Avalia o quão eficiente é cada time ao marcar gols com base na quantidade de posse de bola que teve durante a partida.

Além dessas, outras features são baseadas em diferenças entre métricas importantes como o xG (gols esperados), escanteios, cartões, e pontos por jogo, que ajudam a capturar relações mais complexas e contribuem para melhorar a precisão do modelo na previsão de placares.


In [38]:
# Feature 1: Diferença de posse de bola
df_matches['possession_diff'] = df_matches['home_team_possession'] - df_matches['away_team_possession']

# Feature 2: Proporção de escanteios por posse de bola
df_matches['home_corners_per_possession'] = df_matches['home_team_corner_count'] / df_matches['home_team_possession']
df_matches['away_corners_per_possession'] = df_matches['away_team_corner_count'] / df_matches['away_team_possession']

# Feature 3: Diferença de cartões (amarelos e vermelhos)
df_matches['cards_diff'] = (
    (df_matches['home_team_yellow_cards'] + df_matches['home_team_red_cards']) - 
    (df_matches['away_team_yellow_cards'] + df_matches['away_team_red_cards'])
)

# Adicionar as novas features solicitadas (interações entre as 10 variáveis mais importantes)
df_matches['xg_x_btts'] = df_matches['team_b_xg'] * df_matches['btts_percentage_pre_match']

# Feature 4: Eficiência ofensiva (gols por posse de bola)
df_matches['xg_diff'] = df_matches['team_b_xg'] - df_matches['team_a_xg']
df_matches['home_efficiency_offense'] = df_matches['home_team_goal_count'] / df_matches['home_team_possession']
df_matches['away_efficiency_offense'] = df_matches['away_team_goal_count'] / df_matches['away_team_possession']
df_matches['ppg_diff'] = df_matches['away_ppg'] - df_matches['home_ppg']
df_matches['ppg_x_xg_diff'] = df_matches['ppg_diff'] * df_matches['xg_diff']
df_matches['ppg_x_xg'] = df_matches['ppg_diff'] * df_matches['team_b_xg']
df_matches['corner_diff'] = df_matches['home_team_corner_count'] - df_matches['away_team_corner_count']
df_matches['xg_x_corner'] = df_matches['xg_diff'] * df_matches['corner_diff']

### Visualização 1: Modelo que prevê um único placar para as partidas
### Descrição do Código

O código realiza várias operações para modelagem preditiva de placares em partidas de futebol. Aqui está uma descrição resumida das principais funções e processos:

1. **Função `calcular_gols_recentes`**: Calcula a média de gols marcados por um time nas últimas 5 partidas, permitindo capturar o desempenho recente do time. Isso é feito tanto para o time da casa quanto para o visitante.

2. **Função `calcular_confronto_direto`**: Avalia o histórico de confrontos diretos entre dois times, calculando a média de gols marcados em partidas anteriores entre eles.

3. **Criação de novas features**: 
   - As features como `xg_x_btts`, `ppg_x_btts`, e `corner_x_btts` são geradas para capturar interações entre estatísticas de pré-jogo, como gols esperados (xG) e a probabilidade de ambas as equipes marcarem (BTTS).
   - Várias outras features relacionadas ao desempenho, posse de bola e cartões são incluídas no conjunto de dados.

4. **Treinamento de Modelos Random Forest**: Dois modelos de Random Forest são treinados separadamente para prever os gols do time da casa e do visitante. O conjunto de dados é dividido em treino e teste, e os modelos são treinados com features específicas.

5. **Função `prever_placar_otimizado`**: Esta função usa os modelos treinados para prever o placar de uma partida entre dois times. Ela utiliza dados históricos ou médias das estatísticas de cada time para prever a quantidade de gols, retornando o placar final previsto.

In [39]:
# Função para calcular a média de gols nas últimas 5 partidas para um time
def calcular_gols_recentes(time_encoded, n=5):
    ultimos_jogos = df_matches[df_matches['home_team_encoded'] == time_encoded].tail(n)
    return ultimos_jogos['home_team_goal_count'].mean()

# Adicionar a informação de gols recentes ao dataset para time da casa e visitante
df_matches['home_gols_recentes'] = df_matches['home_team_encoded'].apply(lambda x: calcular_gols_recentes(x))
df_matches['away_gols_recentes'] = df_matches['away_team_encoded'].apply(lambda x: calcular_gols_recentes(x))

# Função para calcular o histórico de confrontos diretos entre os times
def calcular_confronto_direto(home_team_encoded, away_team_encoded):
    confrontos = df_matches[
        (df_matches['home_team_encoded'] == home_team_encoded) & 
        (df_matches['away_team_encoded'] == away_team_encoded)
    ]
    return confrontos['home_team_goal_count'].mean()

# Adicionar feature do histórico de confrontos diretos ao dataset
df_matches['historico_confronto_direto'] = df_matches.apply(
    lambda row: calcular_confronto_direto(row['home_team_encoded'], row['away_team_encoded']), axis=1
)

# Selecionar as mesmas features usadas no treinamento, incluindo as novas
features = [
    'home_team_encoded', 'away_team_encoded', 
    'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 
    'home_ppg', 'away_ppg', 
    'home_team_possession', 'away_team_possession', 
    'home_team_corner_count', 'away_team_corner_count',
    'home_team_yellow_cards', 'away_team_yellow_cards',
    'home_team_red_cards', 'away_team_red_cards',
    'home_gols_recentes', 'away_gols_recentes',  
    'historico_confronto_direto',
    'xg_diff', 'away_efficiency_offense', 'ppg_x_xg_diff', 
    'ppg_x_xg', 'corner_diff', 'xg_x_corner'
]

# Criar o conjunto de dados com as features selecionadas
X = df_matches[features]

# Definir as variáveis alvo (gols do time da casa e visitante)
y_home = df_matches['home_team_goal_count']
y_away = df_matches['away_team_goal_count']

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_home_train, y_home_test = train_test_split(
    X, y_home, test_size=0.2, random_state=42
)

X_train, X_test, y_away_train, y_away_test = train_test_split(
    X, y_away, test_size=0.2, random_state=42
)

# Salvar a lista de features usadas durante o treinamento
train_features = X_train.columns

# Treinar o modelo otimizado de RandomForest com as mesmas features
best_rf_home = RandomForestRegressor(max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=100, random_state=42)
best_rf_away = RandomForestRegressor(max_depth=10, min_samples_leaf=1, min_samples_split=10, n_estimators=100, random_state=42)

# Treinar os modelos com os dados de treino
best_rf_home.fit(X_train, y_home_train)
best_rf_away.fit(X_train, y_away_train)

# Função para prever o placar usando os modelos treinados com as mesmas features
def prever_placar_otimizado(home_team_encoded, away_team_encoded):
    # Selecionar os dados da partida real entre esses dois times, se disponível
    match_data = df_matches[
        (df_matches['home_team_encoded'] == home_team_encoded) &
        (df_matches['away_team_encoded'] == away_team_encoded)
    ]
    
    # Se houver dados de partidas entre esses times, usar esses dados, senão retornar valores médios para predição
    if not match_data.empty:
        input_data = match_data.iloc[0:1].copy()  # Pegar apenas uma linha relevante
    else:
        # Se não houver uma partida direta, usar a média das características de cada time
        team_data_home = df_matches[df_matches['home_team_encoded'] == home_team_encoded].mean(numeric_only=True)
        team_data_away = df_matches[df_matches['away_team_encoded'] == away_team_encoded].mean(numeric_only=True)

        # Criar o input para o modelo baseado nas médias de cada time, garantindo que todas as features usadas no treino estão presentes
        input_data = pd.DataFrame({
            'home_team_encoded': [home_team_encoded],
            'away_team_encoded': [away_team_encoded],
            'Pre-Match PPG (Home)': [team_data_home['Pre-Match PPG (Home)']],
            'Pre-Match PPG (Away)': [team_data_away['Pre-Match PPG (Away)']],
            'home_ppg': [team_data_home['home_ppg']],
            'away_ppg': [team_data_away['away_ppg']],
            'home_team_possession': [team_data_home['home_team_possession']],
            'away_team_possession': [team_data_away['away_team_possession']],
            'home_team_corner_count': [team_data_home['home_team_corner_count']],
            'away_team_corner_count': [team_data_away['away_team_corner_count']],
            'home_team_yellow_cards': [team_data_home['home_team_yellow_cards']],
            'away_team_yellow_cards': [team_data_away['away_team_yellow_cards']],
            'home_team_red_cards': [team_data_home['home_team_red_cards']],
            'away_team_red_cards': [team_data_away['away_team_red_cards']],
            'home_gols_recentes': [team_data_home['home_gols_recentes']],
            'away_gols_recentes': [team_data_away['away_gols_recentes']],
            'historico_confronto_direto': [calcular_confronto_direto(home_team_encoded, away_team_encoded)],
            'xg_diff': [team_data_home['xg_diff']],
            'away_efficiency_offense': [team_data_home['away_efficiency_offense']],
            'home_efficiency_offense': [team_data_home['home_efficiency_offense']],
            'ppg_x_xg_diff': [team_data_home['ppg_x_xg_diff']],
            'corner_diff': [team_data_home['corner_diff']],
            'xg_x_corner': [team_data_home['xg_x_corner']],
        })

    # Alinhar as features da predição com as do treinamento
    input_data = input_data[train_features]

    # Prever os gols do time da casa
    home_goals_pred = best_rf_home.predict(input_data)
    home_goals = round(home_goals_pred[0])

    # Prever os gols do time visitante
    away_goals_pred = best_rf_away.predict(input_data)
    away_goals = round(away_goals_pred[0])

    # Retornar o placar previsto
    return f"Placar previsto: {home_goals} x {away_goals}"

# Testar a função com os modelos otimizados
prever_placar_otimizado(1 , 13)
df_matches.to_csv("../../notebooks/modelo/matches_table.csv")

### Avaliação de métricas
**Predição e Avaliação**
Após o treinamento dos modelos, as predições para o conjunto de teste são realizadas. As métricas usadas para avaliar o desempenho dos modelos são:

**MAE (Erro Médio Absoluto)**: Mede a magnitude média dos erros nas predições.
**MSE (Erro Quadrático Médio)**: Penaliza erros maiores.
**R² (Coeficiente de Determinação)**: Indica o quão bem o modelo se ajusta aos dados de teste. [1]

In [ ]:
# Treinar o modelo otimizado de RandomForest com os melhores parâmetros encontrados
best_rf_home = RandomForestRegressor(
    max_depth=10,               # Melhor max_depth
    min_samples_leaf=1,         # Melhor min_samples_leaf
    min_samples_split=5,        # Melhor min_samples_split
    n_estimators=300,           # Melhor número de árvores
)

best_rf_away = RandomForestRegressor(
    max_depth=10,               # Melhor max_depth
    min_samples_leaf=1,         # Melhor min_samples_leaf
    min_samples_split=5,        # Melhor min_samples_split
    n_estimators=300,           # Melhor número de árvores
)

# Treinar os modelos com os dados de treino
best_rf_home.fit(X_train, y_home_train)
best_rf_away.fit(X_train, y_away_train)

# Prever os resultados no conjunto de teste 
y_home_pred = best_rf_home.predict(X_test)
y_away_pred = best_rf_away.predict(X_test)

# Calcular métricas para as predições de gols do time da casa
mae_home = mean_absolute_error(y_home_test, y_home_pred)
mse_home = mean_squared_error(y_home_test, y_home_pred)
r2_home = r2_score(y_home_test, y_home_pred)

# Calcular métricas para as predições de gols do time visitante
mae_away = mean_absolute_error(y_away_test, y_away_pred)
mse_away = mean_squared_error(y_away_test, y_away_pred)
r2_away = r2_score(y_away_test, y_away_pred)

# Exibir as métricas de avaliação para o time da casa e visitante
print(f"Métricas para o time da casa:")
print(f"MAE: {mae_home}")
print(f"MSE: {mse_home}")
print(f"R²: {r2_home}")

print("\nMétricas para o time visitante:")
print(f"MAE: {mae_away}")
print(f"MSE: {mse_away}")
print(f"R²: {r2_away}")

### Visualização 2: Modelo que prevê os 3 placares mais prováveis de acontecerem em um jogo

#### 1. **Coleta de Dados Históricos**
Os dados históricos das partidas são coletados, e as seguintes features são extraídas:
- **Pre-Match PPG (Home)**: Pontos por jogo do time da casa antes da partida.
- **Pre-Match PPG (Away)**: Pontos por jogo do time visitante antes da partida.
- **away_gols_recentes**: Gols recentes marcados pelo time visitante.
- **away_ppg**: Pontos por jogo do time visitante.
- **home_ppg**: Pontos por jogo do time da casa.
- **home_team_corner_count**: Quantidade de escanteios do time da casa.
- **away_team_corner_count**: Quantidade de escanteios do time visitante.
- **possession_diff**: Diferença de posse de bola entre os dois times.
- **home_corners_per_possession**: Escanteios por posse de bola do time da casa.
- **away_corners_per_possession**: Escanteios por posse de bola do time visitante.
- **cards_diff**: Diferença de cartões (amarelos/vermelhos) entre os dois times.
- **xg_x_btts**: Expected goals (xG) multiplicado pela probabilidade de ambas as equipes marcarem (BTTS).
- **xg_diff**: Diferença de xG entre os times.
- **home_efficiency_offense**: Eficiência ofensiva do time da casa (gols marcados por posse).
- **away_efficiency_offense**: Eficiência ofensiva do time visitante.
- **ppg_diff**: Diferença de PPG entre os dois times.
- **ppg_x_xg_diff**: Produto entre a diferença de PPG e a diferença de xG.
- **corner_diff**: Diferença de escanteios entre os dois times.
- **xg_x_corner**: Produto da diferença de xG e da diferença de escanteios.

#### 2. **Criação da Variável-Alvo (Target)**
A variável-alvo **`match_outcome`** é criada, representando o resultado da partida no formato `home_goals x away_goals`. Isso permite que o modelo seja treinado para prever o placar da partida.

#### 3. **Divisão dos Dados em Conjunto de Treinamento e Teste**
Os dados são divididos em dois conjuntos:
- **Treinamento (80%)**: Usado para treinar o modelo.
- **Teste (20%)**: Usado para avaliar a performance do modelo.

#### 4. **Treinamento do Modelo**
O modelo **RandomForestClassifier** é treinado com as seguintes configurações:
- **n_estimators**: 300 (número de árvores na floresta).
- **max_depth**: 10 (profundidade máxima das árvores).
- **random_state**: 42 (para reprodutibilidade dos resultados).

O modelo é ajustado para aprender padrões nos dados que associam as características dos times ao resultado final da partida.

#### 5. **Previsão dos Três Placares Mais Prováveis**
A função `prever_top_3_placares_futuro(home_team_name, away_team_name)` realiza as seguintes operações:

1. Verifica se os nomes dos times fornecidos estão presentes no dataset.
2. Calcula as médias das features numéricas relevantes para os dois times.
3. Constrói um dicionário com as features usadas no treino, baseadas no histórico dos times fornecidos.
4. Cria um **DataFrame** com essas features e o utiliza como input para o modelo.
5. O modelo retorna as probabilidades associadas a cada possível placar.
6. As três previsões mais prováveis são extraídas, classificadas em ordem decrescente de probabilidade e apresentadas ao usuário.

In [ ]:
from sklearn.ensemble import RandomForestClassifier  # Importação do RandomForestClassifier

# Certifique-se de que as features de treino e de previsão são as mesmas
train_features = [
    'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'away_gols_recentes', 
    'away_ppg', 'away_team_corner_count', 'home_team_corner_count', 
    'home_ppg', 'possession_diff', 'home_corners_per_possession', 
    'away_corners_per_possession', 'cards_diff', 'xg_x_btts', 'xg_diff', 
    'home_efficiency_offense', 'away_efficiency_offense', 'ppg_diff', 
    'ppg_x_xg_diff', 'ppg_x_xg', 'corner_diff', 'xg_x_corner'
]

# Criar a coluna 'match_outcome' com o resultado da partida
df_matches['match_outcome'] = df_matches.apply(lambda row: f"{row['home_team_goal_count']}x{row['away_team_goal_count']}", axis=1)

X = df_matches[train_features]  # Features
y = df_matches['match_outcome']  # Target

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir os parâmetros do modelo
clf = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42)

# Treinar o modelo
clf.fit(X_train, y_train)

# Função para prever os 3 placares mais prováveis usando características históricas dos times
def prever_top_3_placares_futuro(home_team_name, away_team_name):
    # Verificar se os times existem no DataFrame
    if home_team_name not in df_matches['home_team_name'].values:
        print(f"Time da casa '{home_team_name}' não encontrado no dataset.")
        return
    if away_team_name not in df_matches['away_team_name'].values:
        print(f"Time visitante '{away_team_name}' não encontrado no dataset.")
        return
    
    # Filtrar dados históricos do time da casa e do time visitante
    home_team_data = df_matches[df_matches['home_team_name'] == home_team_name].select_dtypes(include='number').mean()
    away_team_data = df_matches[df_matches['away_team_name'] == away_team_name].select_dtypes(include='number').mean()

    # Construir as features baseadas no histórico dos dois times (com features usadas no treino)
    estimated_data = {
        'Pre-Match PPG (Home)': home_team_data['Pre-Match PPG (Home)'],
        'Pre-Match PPG (Away)': away_team_data['Pre-Match PPG (Away)'],
        'away_gols_recentes': away_team_data['away_gols_recentes'],
        'away_ppg': away_team_data['away_ppg'],
        'away_team_corner_count': away_team_data['away_team_corner_count'],
        'home_team_corner_count': home_team_data['home_team_corner_count'],
        'home_ppg': home_team_data['home_ppg'],
        'possession_diff': home_team_data['home_team_possession'] - away_team_data['away_team_possession'],
        'home_corners_per_possession': home_team_data['home_team_corner_count'] / home_team_data['home_team_possession'],
        'away_corners_per_possession': away_team_data['away_team_corner_count'] / away_team_data['away_team_possession'],
        'cards_diff': (home_team_data['home_team_yellow_cards'] + home_team_data['home_team_red_cards']) -
                      (away_team_data['away_team_yellow_cards'] + away_team_data['away_team_red_cards']),
        'xg_x_btts': home_team_data['team_b_xg'] * home_team_data['btts_percentage_pre_match'],
        'xg_diff': away_team_data['team_b_xg'] - home_team_data['team_a_xg'],
        'home_efficiency_offense': home_team_data['home_team_goal_count'] / home_team_data['home_team_possession'],
        'away_efficiency_offense': away_team_data['away_team_goal_count'] / away_team_data['away_team_possession'],
        'ppg_diff': away_team_data['away_ppg'] - home_team_data['home_ppg'],
        'ppg_x_xg_diff': (away_team_data['away_ppg'] - home_team_data['home_ppg']) * (away_team_data['team_b_xg'] - home_team_data['team_a_xg']),
        'ppg_x_xg': (away_team_data['away_ppg'] * away_team_data['team_b_xg']),
        'corner_diff': home_team_data['home_team_corner_count'] - away_team_data['away_team_corner_count'],
        'xg_x_corner': (away_team_data['team_b_xg'] - home_team_data['team_a_xg']) * (home_team_data['home_team_corner_count'] - away_team_data['away_team_corner_count'])
    }

    # Criar um DataFrame para passar como input para o modelo (usando apenas as features do treino)
    input_data = pd.DataFrame([estimated_data])

    # Prever probabilidades para todos os possíveis resultados
    probabilities = clf.predict_proba(input_data)

    # Obter os 3 resultados mais prováveis
    top_3_indices = probabilities[0].argsort()[-3:][::-1]  # Índices das 3 maiores probabilidades
    top_3_outcomes = [clf.classes_[i] for i in top_3_indices]  # Mapeia para os resultados

    # Exibir os resultados
    print(f"Top 3 placares mais prováveis para {home_team_name} vs {away_team_name}:")
    for i, outcome in enumerate(top_3_outcomes, 1):
        print(f"{i}. {outcome}")

    return top_3_outcomes

# Testar a função para uma partida futura
prever_top_3_placares_futuro('Palmeiras', 'Botafogo')

### Otimização com GridSearchCV
Para refinar ainda mais o modelo, GridSearchCV é usado para realizar a busca pelos melhores hiperparâmetros do modelo de Random Forest. O grid testado inclui parâmetros como:

Número de estimadores `(n_estimators)`.
Profundidade máxima `(max_depth)`.
Mínimo de amostras para split `(min_samples_split)` e folhas `(min_samples_leaf)`.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# Parâmetros para ajustar
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Inicializando o RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=param_dist, 
    n_iter=50,  # Número de combinações a testar
    cv=3,  # Validação cruzada
    verbose=2, 
    random_state=42, 
    n_jobs=-1
)

# Ajustando o modelo com RandomizedSearchCV
random_search.fit(X_train, y_away_train)

# Ver os melhores hiperparâmetros encontrados
print("Melhores parâmetros:", random_search.best_params_)

# Testar o modelo com os melhores hiperparâmetros no conjunto de teste
best_rf = random_search.best_estimator_
y_pred_away = best_rf.predict(X_test)

# Avaliar métricas
mae_away = mean_absolute_error(y_away_test, y_pred_away)
mse_away = mean_squared_error(y_away_test, y_pred_away)
r2_away = r2_score(y_away_test, y_pred_away)

print("MAE:", mae_away)
print("MSE:", mse_away)
print("R²:", r2_away)

## Comparação com outros modelos
Além do processo acima, também foi utilizado alguns outros métodos para tentativa da melhor predição possível da partida.

### XGBoost
Um algoritmo de aprendizado supervisionado baseado em boosting, que constrói modelos de forma sequencial, corrigindo erros dos modelos anteriores. O XGBoost é eficiente e conhecido por seu bom desempenho em tarefas de regressão e classificação.

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Função para calcular a média de gols nas últimas 5 partidas para um time
def calcular_gols_recentes(time_encoded, n=5):
    ultimos_jogos = df_matches[df_matches['home_team_encoded'] == time_encoded].tail(n)
    return ultimos_jogos['home_team_goal_count'].mean()

# Adicionar a informação de gols recentes ao dataset para time da casa e visitante
df_matches['home_gols_recentes'] = df_matches['home_team_encoded'].apply(lambda x: calcular_gols_recentes(x))
df_matches['away_gols_recentes'] = df_matches['away_team_encoded'].apply(lambda x: calcular_gols_recentes(x))

# Função para calcular o histórico de confrontos diretos entre os times
def calcular_confronto_direto(home_team_encoded, away_team_encoded):
    confrontos = df_matches[
        (df_matches['home_team_encoded'] == home_team_encoded) & 
        (df_matches['away_team_encoded'] == away_team_encoded)
    ]
    return confrontos['home_team_goal_count'].mean()

# Adicionar feature do histórico de confrontos diretos ao dataset
df_matches['historico_confronto_direto'] = df_matches.apply(
    lambda row: calcular_confronto_direto(row['home_team_encoded'], row['away_team_encoded']), axis=1
)

# Selecionar as mesmas features usadas no treinamento
features = [
    'home_team_encoded', 'away_team_encoded', 
    'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 
    'home_ppg', 'away_ppg', 
    'home_team_possession', 'away_team_possession', 
    'home_team_corner_count', 'away_team_corner_count',
    'home_team_yellow_cards', 'away_team_yellow_cards',
    'home_team_red_cards', 'away_team_red_cards',
    'home_gols_recentes', 'away_gols_recentes',  
    'historico_confronto_direto'  
]

# Criar o conjunto de dados com as features selecionadas
X = df_matches[features]

# Definir as variáveis alvo (gols do time da casa e visitante)
y_home = df_matches['home_team_goal_count']
y_away = df_matches['away_team_goal_count']

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_home_train, y_home_test = train_test_split(
    X, y_home, test_size=0.2, random_state=42
)

X_train, X_test, y_away_train, y_away_test = train_test_split(
    X, y_away, test_size=0.2, random_state=42
)

# Salvar a lista de features usadas durante o treinamento
train_features = X_train.columns

# Treinar o modelo otimizado de XGBoost com as mesmas features
best_xgb_home = XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=10, random_state=42)
best_xgb_away = XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=10, random_state=42)

# Treinar os modelos com os dados de treino
best_xgb_home.fit(X_train, y_home_train)
best_xgb_away.fit(X_train, y_away_train)

# Função para prever o placar usando os modelos treinados com as mesmas features
def prever_placar_otimizado(home_team_encoded, away_team_encoded):
    # Selecionar os dados da partida real entre esses dois times, se disponível
    match_data = df_matches[
        (df_matches['home_team_encoded'] == home_team_encoded) &
        (df_matches['away_team_encoded'] == away_team_encoded)
    ]
    
    # Se houver dados de partidas entre esses times, usar esses dados, senão retornar valores médios para predição
    if not match_data.empty:
        input_data = match_data.iloc[0:1].copy()  # Pegar apenas uma linha relevante
    else:
        # Se não houver uma partida direta, usar a média das características de cada time
        team_data_home = df_matches[df_matches['home_team_encoded'] == home_team_encoded].mean(numeric_only=True)
        team_data_away = df_matches[df_matches['away_team_encoded'] == away_team_encoded].mean(numeric_only=True)

        # Criar o input para o modelo baseado nas médias de cada time, garantindo que todas as features usadas no treino estão presentes
        input_data = pd.DataFrame({
            'home_team_encoded': [home_team_encoded],
            'away_team_encoded': [away_team_encoded],
            'Pre-Match PPG (Home)': [team_data_home['Pre-Match PPG (Home)']],
            'Pre-Match PPG (Away)': [team_data_away['Pre-Match PPG (Away)']],
            'home_ppg': [team_data_home['home_ppg']],
            'away_ppg': [team_data_away['away_ppg']],
            'home_team_possession': [team_data_home['home_team_possession']],
            'away_team_possession': [team_data_away['away_team_possession']],
            'home_team_corner_count': [team_data_home['home_team_corner_count']],
            'away_team_corner_count': [team_data_away['away_team_corner_count']],
            'home_team_yellow_cards': [team_data_home['home_team_yellow_cards']],
            'away_team_yellow_cards': [team_data_away['away_team_yellow_cards']],
            'home_team_red_cards': [team_data_home['home_team_red_cards']],
            'away_team_red_cards': [team_data_away['away_team_red_cards']],
            'home_gols_recentes': [team_data_home['home_gols_recentes']],
            'away_gols_recentes': [team_data_away['away_gols_recentes']],
            'historico_confronto_direto': [calcular_confronto_direto(home_team_encoded, away_team_encoded)]
        })

    # Alinhar as features da predição com as do treinamento
    input_data = input_data[train_features]

    # Prever os gols do time da casa
    home_goals_pred = best_xgb_home.predict(input_data)
    home_goals = round(home_goals_pred[0])

    # Prever os gols do time visitante
    away_goals_pred = best_xgb_away.predict(input_data)
    away_goals = round(away_goals_pred[0])

    # Retornar o placar previsto
    return f"Placar previsto: {home_goals} x {away_goals}"

# Testar a função com os modelos otimizados
prever_placar_otimizado(16, 1)

# Prever os resultados no conjunto de teste 
y_home_pred = best_xgb_home.predict(X_test)
y_away_pred = best_xgb_away.predict(X_test)

# Calcular métricas para as predições de gols do time da casa
mae_home = mean_absolute_error(y_home_test, y_home_pred)
mse_home = mean_squared_error(y_home_test, y_home_pred)
r2_home = r2_score(y_home_test, y_home_pred)

# Calcular métricas para as predições de gols do time visitante
mae_away = mean_absolute_error(y_away_test, y_away_pred)
mse_away = mean_squared_error(y_away_test, y_away_pred)
r2_away = r2_score(y_away_test, y_away_pred)

# Exibir as métricas de avaliação para o time da casa e visitante
print(f"Métricas para o time da casa:")
print(f"MAE: {mae_home}")
print(f"MSE: {mse_home}")
print(f"R²: {r2_home}")

print("\nMétricas para o time visitante:")
print(f"MAE: {mae_away}")
print(f"MSE: {mse_away}")
print(f"R²: {r2_away}")

## Comparação entre XGBoost e Random Forest

Neste documento, será comparado dois dos algoritmos mais populares em Machine Learning: **XGBoost** (eXtreme Gradient Boosting) e **Random Forest**. A comparação será baseada em vários aspectos, como a técnica de aprendizado, desempenho, tempo de treinamento, ajuste de hiperparâmetros e adequação a diferentes tipos de problemas.

---

### 1. Definição e Técnicas Utilizadas

#### Random Forest
O **Random Forest** é um algoritmo de aprendizado de máquina que opera construindo uma série de árvores de decisão durante o treinamento e retornando a média ou a maioria dos resultados (para regressão ou classificação, respectivamente). Ele utiliza **bagging (bootstrap aggregating)**, onde diversas amostras aleatórias são usadas para criar múltiplas árvores independentes. Cada árvore é treinada em uma amostra diferente dos dados de treino.

**Técnicas usadas:**
- **Bagging**: Combina o resultado de várias árvores de decisão treinadas em subconjuntos diferentes dos dados.
- **Diversidade de Árvores**: A aleatoriedade é introduzida selecionando diferentes subconjuntos de amostras e características para cada árvore.
- **Votação/Média**: No caso de classificação, as previsões são decididas por voto majoritário. Em regressão, é calculada a média das previsões.

#### XGBoost
O **XGBoost** é uma implementação eficiente e escalável de **gradient boosting**, uma técnica de aprendizado por conjunto. Ele constrói árvores de decisão de maneira sequencial, onde cada árvore tenta corrigir os erros cometidos pelas árvores anteriores. O XGBoost introduz várias otimizações de hardware e software, tornando-o mais rápido e eficiente em comparação com outros métodos de boosting.

**Técnicas usadas:**
- **Boosting Gradiente**: Adiciona árvores de forma sequencial para corrigir os erros das anteriores.
- **Regularização**: Evita o overfitting aplicando penalizações aos coeficientes das árvores.
- **Poda de Árvore**: Poda de forma inteligente árvores irrelevantes durante o processo de aprendizado.
- **Parallel Processing**: XGBoost utiliza múltiplos núcleos de CPU para acelerar o treinamento.

---

### 2. Desempenho

#### Random Forest
- **Velocidade de Treinamento**: Em geral, o treinamento do Random Forest é mais rápido do que o XGBoost para pequenos conjuntos de dados, mas pode se tornar mais lento à medida que o número de árvores ou a profundidade das árvores aumenta.
- **Desempenho**: O Random Forest tem um desempenho robusto, especialmente em problemas com poucos dados ou com dados ruidosos. Como usa várias árvores independentes, é menos suscetível ao overfitting.
- **Generalização**: Graças ao processo de bagging e ao uso de múltiplas árvores, o Random Forest tende a generalizar bem.

#### XGBoost
- **Velocidade de Treinamento**: XGBoost pode ser mais lento no treinamento inicial devido à natureza sequencial de suas árvores, mas é altamente otimizado para grandes conjuntos de dados e hardware moderno.
- **Desempenho**: XGBoost tende a ser mais preciso em relação ao Random Forest em muitos casos, especialmente em competições de ciência de dados (como no Kaggle), devido à sua capacidade de corrigir erros anteriores e à regularização.
- **Generalização**: XGBoost é mais propenso ao overfitting em pequenos datasets se não for bem regulado, mas com o ajuste correto de parâmetros, ele pode ter excelente generalização.

---

### 3. Ajuste de Hiperparâmetros

#### Random Forest
- O Random Forest é relativamente simples de ajustar. Seus principais hiperparâmetros incluem:
  - **n_estimators**: O número de árvores no modelo.
  - **max_depth**: A profundidade máxima permitida para cada árvore.
  - **min_samples_split**: O número mínimo de amostras necessárias para dividir um nó.
  - **max_features**: O número de características a serem consideradas em cada divisão.
  
  A escolha desses parâmetros geralmente não é muito crítica, pois o modelo é mais robusto a pequenos erros de ajuste, devido à independência das árvores.

#### XGBoost
- XGBoost requer um ajuste de hiperparâmetros mais cuidadoso para maximizar seu desempenho. Os parâmetros mais importantes incluem:
  - **learning_rate**: Controla o peso das árvores adicionadas ao modelo.
  - **n_estimators**: O número de árvores.
  - **max_depth**: A profundidade máxima das árvores.
  - **subsample**: A fração de amostras usadas para treinar cada árvore.
  - **min_child_weight**: Número mínimo de amostras necessárias em um nó folha para fazer uma divisão.
  - **gamma**: Penalidade de regularização para novos nós.

  Ajustar esses parâmetros pode ser mais complexo, mas oferece controle refinado sobre o modelo e pode resultar em um desempenho superior se feito corretamente.

---

### 4. Adequação a Diferentes Cenários

#### Random Forest
- **Vantagens**:
  - Funciona bem com dados de alta dimensionalidade.
  - Bom para conjuntos de dados ruidosos ou que possuem muitos outliers.
  - Simples de ajustar e configurar.
  - Mais robusto para evitar overfitting em comparação com XGBoost.
- **Desvantagens**:
  - Pode ser mais lento com grandes conjuntos de dados.
  - Não capta muito bem relações complexas, pois todas as árvores são independentes.

#### XGBoost
- **Vantagens**:
  - Funciona muito bem em conjuntos de dados grandes e complexos.
  - Permite corrigir erros progressivamente e melhora a precisão geral.
  - Tem um desempenho muito alto em competições de aprendizado de máquina.
  - Oferece regularização para evitar overfitting, mesmo em grandes datasets.
- **Desvantagens**:
  - Mais propenso ao overfitting em pequenos conjuntos de dados se não houver ajuste adequado.
  - Ajuste de hiperparâmetros pode ser mais complicado e demorado.

---

### 5. Casos de Uso

#### Random Forest é mais adequado para:
- **Conjuntos de dados pequenos ou médios** onde a simplicidade e a robustez são mais importantes do que o desempenho ideal.
- Problemas onde os dados têm ruído e outliers.
- **Cenários onde se busca um modelo de baseline** simples e rápido de implementar.

#### XGBoost é mais adequado para:
- **Conjuntos de dados grandes** e **complexos**.
- Cenários onde o desempenho é crítico, como em competições ou projetos de produção de alto desempenho.
- Problemas onde há um **forte foco em precisão** e em **minimizar o erro**, mesmo que isso exija mais tempo para ajuste e treino.

---

### 6. Comparação Geral

| **Critério**               | **Random Forest**                           | **XGBoost**                                |
|----------------------------|---------------------------------------------|--------------------------------------------|
| **Técnica**                 | Bagging                                     | Boosting                                   |
| **Velocidade de Treinamento** | Rápido em conjuntos menores, mas pode ser lento em grandes datasets | Mais lento inicialmente, mas otimizado para grandes datasets |
| **Desempenho**              | Bom desempenho com ajuste mínimo            | Melhor desempenho, mas exige ajuste cuidadoso |
| **Tolerância a Overfitting** | Alta tolerância devido à independência das árvores | Pode overfitar se mal ajustado |
| **Ajuste de Hiperparâmetros** | Relativamente simples                      | Mais complexo e preciso |
| **Melhor uso em**           | Pequenos datasets, baseline simples         | Grandes datasets, precisão máxima, competições |
| **Generalização**           | Geralmente boa, mesmo com ajuste mínimo     | Requer ajuste de regularização para boa generalização |

---

### 7. Conclusão

- **Random Forest** é uma ótima escolha quando se precisa de um modelo robusto que funcione bem com dados variados e não requer muito ajuste de hiperparâmetros.
- **XGBoost** é a escolha ideal para projetos que exigem o melhor desempenho possível, especialmente quando há dados grandes e complexos e pode investir tempo em ajustar os parâmetros.

Se um modelo rápido e eficaz com uma boa base de desempenho sem muita complicação é o ideal, **Random Forest** é a escolha mais segura. Por outro lado, se o objetivo é maximizar a precisão e há tempo e recursos para ajustar hiperparâmetros, **XGBoost** geralmente proporciona um desempenho superior.


## Referências
Está é uma seção de referências com relação as bibliotecas que utilizamos ao longo deste arquivo

[1] Júnior, Clébio de Oliveira. “Prevendo Números: Entendendo as Métricas R2, MAE, MAPE, MSE E RMSE.” Data Hackers, 13 Dec. 2021, medium.com/data-hackers/prevendo-n%C3%BAmeros-entendendo-m%C3%A9tricas-de-regress%C3%A3o-35545e011e70.